In [ ]:
%load_ext sql


Make sure to switch to retail_dm database to create external schema to query Glue Catalog Tables using Redshift Spectrum.

In [ ]:
%%sql

CREATE EXTERNAL SCHEMA IF NOT EXISTS retail_db
FROM DATA CATALOG
DATABASE 'retail_db'
IAM_ROLE 'arn:aws:iam::269066542444:role/AIRedshiftRetailSpectrum'
CREATE EXTERNAL DATABASE IF NOT EXISTS;

In [ ]:
%%sql

SELECT *
FROM retail_db.orders
LIMIT 10;

In [ ]:
%%sql

SELECT *
FROM retail_db.order_items
LIMIT 10;

In [ ]:
%%sql

DROP TABLE IF EXISTS daily_product_revenue;

In [ ]:
%%sql

CREATE TABLE public.daily_product_revenue
DISTKEY (1)
SORTKEY (1, 4)
AS
SELECT o.order_date,
    o.order_status,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM retail_db.orders AS o
    JOIN retail_db.order_items AS oi
        ON o.order_id = oi.order_item_order_id
GROUP BY o.order_date,
    o.order_status,
    oi.order_item_product_id
ORDER BY o.order_date, 4 DESC;

In [ ]:
%%sql

SELECT count(*) FROM daily_product_revenue;

In [ ]:
%%sql

SELECT * FROM daily_product_revenue 
ORDER BY order_date, revenue DESC
LIMIT 10;